# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json` <b>(I will be using pymongo and json to import the data into our MongoDB service.)</b>

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import json
import pandas as pd
import os

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Create 'uk_food' Database and assign variable
uk_food = mongo['uk_food']

# Create establishment collection and assign variable
establishments = uk_food['establishments']

# Load establishments.json data into establishments collection
with open(os.path.join('Resources', 'establishments.json')) as file:
    file_data = json.load(file)

establishments.insert_many(file_data)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'demoDB', 'local', 'uk_food']


In [5]:
# review the collections in our new database
print(uk_food.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
doc = establishments.find().limit(1)
pprint(doc[0])

print(f'There are {"{:,}".format(establishments.count_documents({}))} documents in the collection')

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64360ac649d0e543363d3621'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [7]:
# Create a dictionary for the new restaurant data
new_restaurant = {
                    "BusinessName":"Penang Flavours",
                    "BusinessType":"Restaurant/Cafe/Canteen",
                    "BusinessTypeID":"",
                    "AddressLine1":"Penang Flavours",
                    "AddressLine2":"146A Plumstead Rd",
                    "AddressLine3":"London",
                    "AddressLine4":"",
                    "PostCode":"SE18 7DY",
                    "Phone":"",
                    "LocalAuthorityCode":"511",
                    "LocalAuthorityName":"Greenwich",
                    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
                    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
                    "scores":{
                        "Hygiene":"",
                        "Structural":"",
                        "ConfidenceInManagement":""
                    },
                    "SchemeType":"FHRS",
                    "geocode":{
                        "longitude":"0.08384000",
                        "latitude":"51.49014200"
                    },
                    "RightToReply":"",
                    "Distance":4623.9723280747176,
                    "NewRatingPending":True
                                                }

In [8]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [9]:
# Check that the new restaurant was inserted
pprint(establishments.find(new_restaurant)[0])
print(f'There are {"{:,}".format(establishments.count_documents({}))} documents in the collection')

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64360ac749d0e543363dd184'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}
There are 39,780 documents in the collection


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [10]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType' : 'Restaurant/Cafe/Canteen'}
fields = {
          'BusinessType': 1,
          'BusinessTypeID': 1}

type_id = establishments.find(query, fields)[0]['BusinessTypeID']
pprint(establishments.find(query, fields)[0])
print(type_id)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('64360ac649d0e543363d3621')}
1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [11]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName": "Penang Flavours"}, { '$set' : {'BusinessTypeID' : type_id}})

In [12]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64360ac749d0e543363dd184'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [13]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName' : 'Dover'}

print(f'There are {"{:,}".format(establishments.count_documents({}))} documents in the collection')
print(f"There are {establishments.count_documents(query)} documents where LocalAuthorityName is 'Dover'")

There are 39,780 documents in the collection
There are 994 documents where LocalAuthorityName is 'Dover'


In [14]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [15]:
# Check if any remaining documents include Dover
print(f"There are {establishments.count_documents(query)} documents where LocalAuthorityName is 'Dover'")

There are 0 documents where LocalAuthorityName is 'Dover'


In [16]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())
print(f'There are still {"{:,}".format(establishments.count_documents({}))} documents in the collection')

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64360ac649d0e543363d3908'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [17]:
# Change the data type from String to Decimal for latitude and longitude
establishments.update_many({}, [{'$set': { "geocode.latitude" : {'$toDouble': '$geocode.latitude' },
                                         "geocode.longitude": {'$toDouble':'$geocode.longitude'}}}])

In [18]:
# Check that the coordinates are now numbers
result = establishments.find_one({},{'geocode': 1})
pprint(result['geocode'])
print(f"'latitude' dtype = {type(result['geocode']['latitude'])}")
print(f"'longitude' dtype = {type(result['geocode']['latitude'])}")

{'latitude': 51.083812, 'longitude': 1.195625}
'latitude' dtype = <class 'float'>
'longitude' dtype = <class 'float'>


In [19]:
# Drop Database for Jupyter Notebook Reruns
# mongo.drop_database('uk_food')